In [78]:
import nltk
from nltk.corpus import brown
from collections import Counter
brown_words = brown.words()
unigram_counts = Counter(brown_words)
bigrams = []
for sent in brown.sents():
    bigrams.extend(nltk.bigrams(sent, pad_left=True, pad_right=True))
bigram_counts = Counter(bigrams)

<h3>Language modeling</h3>

Another(!) very common problem in NLP:
<center>
<p style="border:3px; width: 75%; border-radius: 25px; background-color:lightgrey; border-style:solid; border-color:black; padding: 1em;">
<i>how likely is that a sequence of words comes from a particular language (e.g. English)?
</i>
</p>
</center>

<b>Odd problem. Applications?</b>

- speech recognition
- machine translation
- grammatical error detection

### Language modeling with the perceptron?

Maybe:
- we have a lot of data: all the English Web
- we don't have labels
- can assume everything we see is "good English"; negative instances?
- we need probabilities; less probable not ungrammatical

Let't think about it differently.

<h3>Problem setup</h3>

Training data is a (large) set of sentences $\mathbf{x}^m$ with words $x_n$:

<p>
\begin{align}
D_{train} & = \{\mathbf{x}^1,...,\mathbf{x}^M\} \\
\mathbf{x}& = [x_1,... x_N]\\
\end{align}
</p>

<p class="fragment">
for example:
\begin{align}
\mathbf{x}=&[\text{The}, \text{water}, \text{is}, \text{clear}, \text{.}]
\end{align}
</p>

<p class="fragment">We want to learn a model that returns:
\begin{align}
\text{probability}\; P(\mathbf{x}), \mathbf{for} \; \forall \mathbf{x}\in V^{maxN}
\end{align}
$V$ is the vocabulary and $V^{maxN}$ all possible sentences
</p>

### Word counts

Let's take a corpus:

In [5]:
brown.sents() 

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

Count its words:

In [47]:
print(len(brown_words))

1161192


Count the occurrences of one word:

In [56]:
print(unigram_counts["the"])

62713


### Word counts to probabilities

probability $P(\text{the}) = \frac{counts(the)}{\sum_{x \in Vocabulary} counts(x)}$

In [57]:
print(unigram_counts["the"]/len(brown_words))

0.05400743374050114


To have a probability distributions and not just scores, the sum of the probabilities for all words must be 1. Let's check:

In [61]:
sum = 0
for word in unigram_counts:
    sum += unigram_counts[word]
print(sum/len(brown_words))

1.0


### Unigram language model

Multiply the probability of each word in the sentence $\mathbf{x}$:

$$P(\mathbf{x}) = \prod_{n=1}^N  P(x_n) = \prod_{n=1}^N \frac{counts(x_n)}{\sum_{x \in V} counts(x)}$$

Our first language model! What could go wrong?

the most probable word is "the"
- the most probable single-word sentence is "the"
- the most probable two-word sentence is "the the"
- the most probable $N$-word sentence is $N\times $"the"

Sentences with words not seen in the training data have 0 probability.

In [60]:
print(unigram_counts["genome"]/len(brown_words))

0.0


<h3>On the way to a better language model</h3>

Instead of:

$$P(\mathbf{x}) = \prod_{n=1}^N  P(x_n)$$

Let's assume that the choice of a word depends on the one before it:

$$P(\mathbf{x}) = \prod_{n=1}^N P(x_n| x_{n-1})$$

The probability of a word $x_n$ following another $x_{n-1}$ is:

$$P(x_n| x_{n-1})=\frac{bigram\_counts(x_{n-1}, x_n)}{counts(x_{n-1})}$$

This is the **bigram** language model.

### Bigram Language model in action

$$\mathbf{x}=[\text{The}, \text{water}, \text{is}, \text{clear}, \text{.}]$$

\begin{align}
P(\mathbf{x}) =& P(\text{The}|\text{None})\times P(\text{water}|\text{The})\times P\text{is}|\text{water})\times\\
&P(\text{clear}|\text{is})\times P(\text{.}|\text{clear})\times P\text{.}|\text{None})
\end{align}

In [93]:
x = ["The", "water", "is", "clear", "."]
x_bigrams = nltk.bigrams(x, pad_left=True, pad_right=True)
prob_x = 1.0
for bg in x_bigrams:
    if bg[0] == None:
        prob_bg = bigram_counts[bg]/len(brown.sents())
    else:
        prob_bg = bigram_counts[bg]/unigram_counts[bg[0]]
    prob_x = prob_x *prob_bg
    print(str(bg)+":"+str(prob_bg))
print(prob_x)

(None, 'The'):0.11412626438786187
('The', 'water'):0.0009644530173601543
('water', 'is'):0.030162412993039442
('is', 'clear'):0.002497253021676156
('clear', '.'):0.091324200913242
('.', None):1.0
7.571487129955549e-10


Higher order n-gram language models are possible and are used with very large datasets (5-grams with 1B tokens are common)

Evaluation 
there is perplexity but...
with sentence completion/error correction